# Your First AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) from Oxford of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load the image dataset and create a pipeline.
* Build and Train an image classifier on this dataset.
* Use your trained model to perform inference on flower images.

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

In [1]:
# %pip uninstall tensorflow_datasets y
# %pip install "keras==2.3.0"
# %pip install "keras==2.3.0"
# %pip install "tensorflow_datasets==3.1.0"
%pip --no-cache-dir install tfds-nightly

Note: you may need to restart the kernel to use updated packages.


## Import Resources

In [1]:
# TODO: Make all necessary imports.
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [2]:
print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 tf.keras version:', tf.keras.__version__)
print('\t\u2022 tfds version:', tfds.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

Using:
	• TensorFlow version: 2.2.0-rc4
	• tf.keras version: 2.3.0-tf
	• tfds version: 3.1.0
	• GPU device not found. Running on CPU


## Load the Dataset

Here you'll use `tensorflow_datasets` to load the [Oxford Flowers 102 dataset](https://www.tensorflow.org/datasets/catalog/oxford_flowers102). This dataset has 3 splits: `'train'`, `'test'`, and `'validation'`.  You'll also need to make sure the training data is normalized and resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet, but you'll still need to normalize and resize the images to the appropriate size.

In [6]:
dataset, dataset_info = tfds.load('oxford_flowers102', as_supervised=True, with_info=True)


NonMatchingChecksumError: Artifact https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz, downloaded to /Users/akanshasinghlnu/tensorflow_datasets/downloads/robots.ox.ac.uk_vgg_flowers_102_102flowersoWedSp98maBn1wypsDib6T-q2NVbO40fwvTflmPmQpY.tgz.tmp.63b75e4b6049419c9f015ea6f8527110/102flowers.tgz, has wrong checksum. This might indicate:
 * The website may be down (e.g. returned a 503 status code). Please check the url.
 * For Google Drive URLs, try again later as Drive sometimes rejects downloads when too many people access the same URL. See https://github.com/tensorflow/datasets/issues/1482
 * The original datasets files may have been updated. In this case the TFDS dataset builder should be updated to use the new files and checksums. Sorry about that. Please open an issue or send us a PR with a fix.
 * If you're adding a new dataset, don't forget to register the checksums as explained in: https://www.tensorflow.org/datasets/add_dataset#2_run_download_and_prepare_locally


In [3]:
train_split = 60
test_val_split = 20

splits = tfds.Split.ALL.subsplit([train_split, test_val_split, test_val_split])

AttributeError: type object 'Split' has no attribute 'ALL'

In [ ]:
!python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums=True --datasets=oxford_flowers102

I0505 10:16:07.629642 4603192768 download_and_prepare.py:194] Running download_and_prepare for dataset(s):
oxford_flowers102
I0505 10:16:07.802696 4603192768 dataset_info.py:430] Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: oxford_flowers102/2.1.0
I0505 10:16:08.813424 4603192768 dataset_info.py:361] Load dataset info from /var/folders/3f/zmh1hs5j2vv76bml_5h7h0zc0000gn/T/tmp7a5trpwvtfds
I0505 10:16:08.815791 4603192768 dataset_info.py:401] Field info.description from disk and from code do not match. Keeping the one from code.
I0505 10:16:08.815922 4603192768 dataset_info.py:401] Field info.citation from disk and from code do not match. Keeping the one from code.
I0505 10:16:08.816222 4603192768 download_and_prepare.py:133] download_and_prepare for dataset oxford_flowers102/2.1.0...
I0505 10:16:08.816561 4603192768 dataset_builder.py:333] Generating dataset oxford_flowers102 (/Users/akanshasinghlnu/tensorflow_datasets/oxford_flowers102/2.1.0)
Dl Completed...: 0

Extraction completed...: 0 file [00:12, ? file/s]
Dl Size...:   7%|█▉                          | 23/328 [00:12<02:24,  2.11 MiB/s]

Extraction completed...: 0 file [00:12, ? file/s]
Dl Size...:   7%|██                          | 24/328 [00:13<02:24,  2.10 MiB/s]

Extraction completed...: 0 file [00:13, ? file/s]
Dl Size...:   8%|██▏                         | 25/328 [00:13<02:29,  2.03 MiB/s]

Extraction completed...: 0 file [00:13, ? file/s]
Dl Size...:   8%|██▏                         | 26/328 [00:14<02:29,  2.02 MiB/s]

Extraction completed...: 0 file [00:14, ? file/s]
Dl Size...:   8%|██▎                         | 27/328 [00:14<02:28,  2.02 MiB/s]

Extraction completed...: 0 file [00:14, ? file/s]
Dl Size...:   9%|██▍                         | 28/328 [00:15<02:26,  2.05 MiB/s]

Extraction completed...: 0 file [00:15, ? file/s]
Dl Size...:   9%|██▍                         | 29/328 [00:15<02:14,  2.23 MiB/s]

Extraction completed...: 0 file [00:15, ? file/s]
Dl Size...:   9%|██▌      

Extraction completed...: 0 file [00:29, ? file/s]
Dl Size...:  18%|█████                       | 60/328 [00:30<02:06,  2.12 MiB/s]

Extraction completed...: 0 file [00:30, ? file/s]
Dl Size...:  19%|█████▏                      | 61/328 [00:30<02:06,  2.12 MiB/s]

Extraction completed...: 0 file [00:30, ? file/s]
Dl Size...:  19%|█████▎                      | 62/328 [00:31<02:05,  2.12 MiB/s]

Extraction completed...: 0 file [00:31, ? file/s]
Dl Size...:  19%|█████▍                      | 63/328 [00:31<02:05,  2.12 MiB/s]

Extraction completed...: 0 file [00:31, ? file/s]
Dl Size...:  20%|█████▍                      | 64/328 [00:32<02:05,  2.11 MiB/s]

Extraction completed...: 0 file [00:32, ? file/s]
Dl Size...:  20%|█████▌                      | 65/328 [00:32<02:04,  2.12 MiB/s]

Extraction completed...: 0 file [00:32, ? file/s]
Dl Size...:  20%|█████▋                      | 66/328 [00:33<02:04,  2.11 MiB/s]

Extraction completed...: 0 file [00:33, ? file/s]
Dl Size...:  20%|█████▋   

Extraction completed...: 0 file [00:47, ? file/s]
Dl Size...:  30%|████████▎                   | 97/328 [00:47<01:48,  2.12 MiB/s]

Extraction completed...: 0 file [00:47, ? file/s]
Dl Size...:  30%|████████▎                   | 98/328 [00:48<01:48,  2.12 MiB/s]

Extraction completed...: 0 file [00:48, ? file/s]
Dl Size...:  30%|████████▍                   | 99/328 [00:48<01:47,  2.12 MiB/s]

Extraction completed...: 0 file [00:48, ? file/s]
Dl Size...:  30%|████████▏                  | 100/328 [00:49<01:47,  2.12 MiB/s]

Extraction completed...: 0 file [00:49, ? file/s]
Dl Size...:  31%|████████▎                  | 101/328 [00:49<01:47,  2.12 MiB/s]

Extraction completed...: 0 file [00:49, ? file/s]
Dl Size...:  31%|████████▍                  | 102/328 [00:50<01:46,  2.12 MiB/s]

Extraction completed...: 0 file [00:50, ? file/s]
Dl Size...:  31%|████████▍                  | 103/328 [00:50<01:46,  2.11 MiB/s]

Extraction completed...: 0 file [00:50, ? file/s]
Dl Size...:  32%|████████▌

Extraction completed...: 0 file [01:04, ? file/s]
Dl Size...:  41%|███████████                | 134/328 [01:05<01:31,  2.12 MiB/s]

Extraction completed...: 0 file [01:05, ? file/s]
Dl Size...:  41%|███████████                | 135/328 [01:05<01:31,  2.12 MiB/s]

Extraction completed...: 0 file [01:05, ? file/s]
Dl Size...:  41%|███████████▏               | 136/328 [01:06<01:30,  2.11 MiB/s]

Extraction completed...: 0 file [01:06, ? file/s]
Dl Size...:  42%|███████████▎               | 137/328 [01:06<01:30,  2.11 MiB/s]

Extraction completed...: 0 file [01:06, ? file/s]
Dl Size...:  42%|███████████▎               | 138/328 [01:07<01:29,  2.12 MiB/s]

Extraction completed...: 0 file [01:07, ? file/s]
Dl Size...:  42%|███████████▍               | 139/328 [01:07<01:29,  2.11 MiB/s]

Extraction completed...: 0 file [01:07, ? file/s]
Dl Size...:  43%|███████████▌               | 140/328 [01:08<01:28,  2.11 MiB/s]

Extraction completed...: 0 file [01:08, ? file/s]
Dl Size...:  43%|█████████

Extraction completed...: 0 file [01:22, ? file/s]
Dl Size...:  52%|██████████████             | 171/328 [01:22<01:14,  2.12 MiB/s]

Extraction completed...: 0 file [01:22, ? file/s]
Dl Size...:  52%|██████████████▏            | 172/328 [01:23<01:13,  2.12 MiB/s]

Extraction completed...: 0 file [01:23, ? file/s]
Dl Size...:  53%|██████████████▏            | 173/328 [01:23<01:13,  2.12 MiB/s]

Extraction completed...: 0 file [01:23, ? file/s]
Dl Size...:  53%|██████████████▎            | 174/328 [01:24<01:12,  2.12 MiB/s]

Extraction completed...: 0 file [01:24, ? file/s]
Dl Size...:  53%|██████████████▍            | 175/328 [01:24<01:12,  2.12 MiB/s]

Extraction completed...: 0 file [01:24, ? file/s]
Dl Size...:  54%|██████████████▍            | 176/328 [01:25<01:11,  2.12 MiB/s]

Extraction completed...: 0 file [01:25, ? file/s]
Dl Size...:  54%|██████████████▌            | 177/328 [01:25<01:11,  2.12 MiB/s]

Extraction completed...: 0 file [01:25, ? file/s]
Dl Size...:  54%|█████████

In [8]:
# test_split, valid_split, train_split = tfds.Split.Train([10, 15, 75])
first_67_percent = tfds.Split.TRAIN.subsplit(tfds.percent[:67])
# test_set = tfds.load("oxford_flowers102", split=test_split, as_supervised=True)

AttributeError: 'Split' object has no attribute 'subsplit'

In [6]:
# TODO: Load the dataset with TensorFlow Datasets.
train_split = 60
test_val_split = 20

# splits = tfds.Split.ALL.subsplit([train_split, test_val_split, test_val_split])
# splits = tfds.Split.TRAIN.subsplit([train_split, test_val_split, test_val_split])
dataset, dataset_info = tfds.load('oxford_flowers102', data_dir='/Users/akanshasinghlnu/tensorflow_datasets', as_supervised=True, with_info=True)


# TODO: Create a training set, a validation set and a test set.
# training_set, validation_set, test_set = dataset


/Users/akanshasinghlnu/tensorflow_datasets/oxford_flowers102/2.1.1
Using /Users/akanshasinghlnu/tensorflow_datasets/oxford_flowers102/2.1.0 instead.


NonMatchingChecksumError: Artifact https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz, downloaded to /Users/akanshasinghlnu/tensorflow_datasets/downloads/robots.ox.ac.uk_vgg_flowers_102_102flowersoWedSp98maBn1wypsDib6T-q2NVbO40fwvTflmPmQpY.tgz.tmp.698515780ccc49a4b3c890a51063639a/102flowers.tgz, has wrong checksum. This might indicate:
 * The website may be down (e.g. returned a 503 status code). Please check the url.
 * For Google Drive URLs, try again later as Drive sometimes rejects downloads when too many people access the same URL. See https://github.com/tensorflow/datasets/issues/1482
 * The original datasets files may have been updated. In this case the TFDS dataset builder should be updated to use the new files and checksums. Sorry about that. Please open an issue or send us a PR with a fix.
 * If you're adding a new dataset, don't forget to register the checksums as explained in: https://www.tensorflow.org/datasets/add_dataset#2_run_download_and_prepare_locally


In [11]:
dataset.info

{'test': <PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>,
 'train': <PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>,
 'validation': <PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>}

## Explore the Dataset

In [9]:
# TODO: Get the number of examples in each set from the dataset info.
print (dataset_info.total_num_examples)
# TODO: Get the number of classes in the dataset from the dataset info.


AttributeError: 'DatasetInfo' object has no attribute 'total_num_examples'

In [ ]:
# TODO: Print the shape and corresponding label of 3 images in the training set.


In [ ]:
# TODO: Plot 1 image from the training set. Set the title 
# of the plot to the corresponding image label. 


### Label Mapping

You'll also need to load in a mapping from label to category name. You can find this in the file `label_map.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/3.7/library/json.html). This will give you a dictionary mapping the integer coded labels to the actual names of the flowers.

In [ ]:
with open('label_map.json', 'r') as f:
    class_names = json.load(f)

In [ ]:
# TODO: Plot 1 image from the training set. Set the title 
# of the plot to the corresponding class name. 



## Create Pipeline

In [ ]:
# TODO: Create a pipeline for each set.



# Build and Train the Classifier

Now that the data is ready, it's time to build and train the classifier. You should use the MobileNet pre-trained model from TensorFlow Hub to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! 

Refer to the rubric for guidance on successfully completing this section. Things you'll need to do:

* Load the MobileNet pre-trained network from TensorFlow Hub.
* Define a new, untrained feed-forward network as a classifier.
* Train the classifier.
* Plot the loss and accuracy values achieved during training for the training and validation set.
* Save your trained model as a Keras model. 

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right.

**Note for Workspace users:** One important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module. Also, If your model is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# TODO: Build and train your network.



In [ ]:
# TODO: Plot the loss and accuracy values achieved during training for the training and validation set.



## Testing your Network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Print the loss and accuracy values achieved on the entire test set.



## Save the Model

Now that your network is trained, save the model so you can load it later for making inference. In the cell below save your model as a Keras model (*i.e.* save it as an HDF5 file).

In [ ]:
# TODO: Save your trained model as a Keras model.



## Load the Keras Model

Load the Keras model you saved above.

In [ ]:
# TODO: Load the Keras model



# Inference for Classification

Now you'll write a function that uses your trained network for inference. Write a function called `predict` that takes an image, a model, and then returns the top $K$ most likely class labels along with the probabilities. The function call should look like: 

```python
probs, classes = predict(image_path, model, top_k)
```

If `top_k=5` the output of the `predict` function should be something like this:

```python
probs, classes = predict(image_path, model, 5)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

Your `predict` function should use `PIL` to load the image from the given `image_path`. You can use the [Image.open](https://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.open) function to load the images. The `Image.open()` function returns an `Image` object. You can convert this `Image` object to a NumPy array by using the `np.asarray()` function.

The `predict` function will also need to handle pre-processing the input image such that it can be used by your model. We recommend you write a separate function called `process_image` that performs the pre-processing. You can then call the `process_image` function from the `predict` function. 

### Image Pre-processing

The `process_image` function should take in an image (in the form of a NumPy array) and return an image in the form of a NumPy array with shape `(224, 224, 3)`.

First, you should convert your image into a TensorFlow Tensor and then resize it to the appropriate size using `tf.image.resize`.

Second, the pixel values of the input images are typically encoded as integers in the range 0-255, but the model expects the pixel values to be floats in the range 0-1. Therefore, you'll also need to normalize the pixel values. 

Finally, convert your image back to a NumPy array using the `.numpy()` method.

In [ ]:
# TODO: Create the process_image function



To check your `process_image` function we have provided 4 images in the `./test_images/` folder:

* cautleya_spicata.jpg
* hard-leaved_pocket_orchid.jpg
* orange_dahlia.jpg
* wild_pansy.jpg

The code below loads one of the above images using `PIL` and plots the original image alongside the image produced by your `process_image` function. If your `process_image` function works, the plotted image should be the correct size. 

In [ ]:
from PIL import Image

image_path = './test_images/hard-leaved_pocket_orchid.jpg'
im = Image.open(image_path)
test_image = np.asarray(im)

processed_test_image = process_image(test_image)

fig, (ax1, ax2) = plt.subplots(figsize=(10,10), ncols=2)
ax1.imshow(test_image)
ax1.set_title('Original Image')
ax2.imshow(processed_test_image)
ax2.set_title('Processed Image')
plt.tight_layout()
plt.show()

Once you can get images in the correct format, it's time to write the `predict` function for making inference with your model.

### Inference

Remember, the `predict` function should take an image, a model, and then returns the top $K$ most likely class labels along with the probabilities. The function call should look like: 

```python
probs, classes = predict(image_path, model, top_k)
```

If `top_k=5` the output of the `predict` function should be something like this:

```python
probs, classes = predict(image_path, model, 5)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

Your `predict` function should use `PIL` to load the image from the given `image_path`. You can use the [Image.open](https://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.open) function to load the images. The `Image.open()` function returns an `Image` object. You can convert this `Image` object to a NumPy array by using the `np.asarray()` function.

**Note:** The image returned by the `process_image` function is a NumPy array with shape `(224, 224, 3)` but the model expects the input images to be of shape `(1, 224, 224, 3)`. This extra dimension represents the batch size. We suggest you use the `np.expand_dims()` function to add the extra dimension. 

In [ ]:
# TODO: Create the predict function



# Sanity Check

It's always good to check the predictions made by your model to make sure they are correct. To check your predictions we have provided 4 images in the `./test_images/` folder:

* cautleya_spicata.jpg
* hard-leaved_pocket_orchid.jpg
* orange_dahlia.jpg
* wild_pansy.jpg

In the cell below use `matplotlib` to plot the input image alongside the probabilities for the top 5 classes predicted by your model. Plot the probabilities as a bar graph. The plot should look like this:

<img src='assets/inference_example.png' width=600px>

You can convert from the class integer labels to actual flower names using `class_names`. 

In [ ]:
# TODO: Plot the input image along with the top 5 classes
